# Hybrid RAG

In [1]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langsmith.run_helpers import traceable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter
import uuid
import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare.data_load import DocDataLoader
from src.rag.types import RAGResponse, RAGCategories

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class HybridGeminiRag:
    """
        Formula: RAG = Gemini + ES + Hybrid Search
        Features:
        - Vector Store Memory
    """

    def __init__(self, es_index: str, embed_model: str, config: Configuration):
        """
            Initialize the RAG
            @param es_index elastic search index
            @param embed_model hugging face sentence-transformer model
            @param config Configuration
        """
        self.chat_model = ChatGoogleGenerativeAI(
            model="gemini-pro", 
            temperature=0, 
            google_api_key=config.load_gemini_token()
        )
        self.ask_memory = {}
        template = ""
        template += "You are a helpful admission assistant for Ton Duc Thang university who gives helpful and accurate information.\n"
        template += 'Answer the question mainly using the following context. Output "None" if you cannot answer:'
        template += "\n```\n{context}\n```\n"
        template += "Question: {question}\n"
        prompt = ChatPromptTemplate.from_template(template)
        self._init_retriever(es_index=es_index, embed_model=embed_model, config=config)
        self.rag = (
            {"context": itemgetter("question") | self.ensemble_retriever, 
             "question": itemgetter("question"), "ask_id": itemgetter("ask_id")}
            | RunnableLambda(self._format_doc)
            | prompt
            | self.chat_model
            | StrOutputParser()
        ) 
        return
    
    def _format_doc(self, input_dict):
        retrieve_doc = input_dict['context'][0]
        doc_id = retrieve_doc.metadata['id']
        document = self.database[doc_id]
        # store to a temporary mem
        self.ask_memory[input_dict['ask_id']] = document
        input_dict['context'] = document.content
        return input_dict

    @traceable(tags=['hybrid_gemini'])
    def ask_rag(self, question: str) -> RAGResponse:
        """
            Ask the RAG a question.
            @param question

            @return answer, 'None' if it can't
        """
        ask_id = uuid.uuid1().int
        if question == None:
            return "No question was asked"
        if ask_id == None:
            return "Went wrong. Cannot generate uuid"
        answer = self.rag.invoke({"question": question, "ask_id": ask_id})
        return RAGResponse(answer=answer.strip(), category=RAGCategories.major, document=self.ask_memory.pop(ask_id))

    def _init_retriever(self, es_index:str, embed_model:str, config:Configuration):
        """
            Initialize Elastic, BM25 instances and combine them into hybrid search
            This method is intented for private use. Please be cautious modifying.
        """
        embeddings = HuggingFaceInferenceAPIEmbeddings(
            model_name=embed_model, api_key=config.load_hg_token())
        # elastic
        es = ElasticsearchStore(
            es_connection=config.load_elasticsearch_connection(),
            embedding=embeddings,
            index_name=es_index, distance_strategy="EUCLIDEAN_DISTANCE") 
        self.es_retriever = es.as_retriever(search_kwargs={"k": 1})
        # BM-25
        doc_loader = DocDataLoader()
        db = doc_loader.load_major_docs(size=460, overlap=20)
        self.database = doc_loader.load_major_docs_full_asmap()
        self.bm25_retriever = BM25Retriever.from_documents(db)
        self.bm25_retriever.k = 1
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.es_retriever],
            weights=[0.5, 0.5])
        del doc_loader
        return 


In [11]:
config = Configuration()
rag = HybridGeminiRag("labse-major", "sentence-transformers/LaBSE", config)

config.enable_tracing(project="LEARN")

From (44, 4) to Counter({2: 37, 1: 6, 3: 1})
From (44, 4) to 44


# My code

In [1]:
# import
import sys
sys.path.append("../")

from src.rag.hybrid_rag import HybridGeminiRAG
from src.service.provider import ProviderService
services = ProviderService()

major = services.get_categories().major
rag = HybridGeminiRAG(provider=services, rag_config=major, update_notification_func=lambda x: x)


/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 5) to 44
From (11, 4) to 11


In [2]:
q = "cho em hỏi về ngành phần mềm"
a = rag.ask_rag(question=q)
a

2
cho em hỏi về ngành phần mềm


In [4]:
a.answer

'**Ngành Kỹ thuật Phần mềm tại Trường Đại học Tôn Đức Thắng**\n\n**Giới thiệu**\n\nNgành Kỹ thuật Phần mềm ứng dụng kiến thức về lập trình, thiết kế phần mềm, quy trình vận hành và phát triển phần mềm, phân tích yêu cầu hệ thống, trí tuệ nhân tạo... để tạo ra các sản phẩm phần mềm chất lượng cao đáp ứng nhu cầu của xã hội.\n\n**Cơ hội nghề nghiệp**\n\nSinh viên tốt nghiệp ngành Kỹ thuật Phần mềm có thể làm việc tại các vị trí sau:\n\n* Chuyên viên phân tích yêu cầu (BA)\n* Kiến trúc sư phần mềm (SA)\n* Lập trình viên (Developer)\n* Chuyên viên thử nghiệm phần mềm (Tester)\n* Chuyên viên cầu nối (BrSE)\n* Chuyên viên nghiên cứu và phát triển ứng dụng (R&D)\n* Giảng viên giảng dạy các môn học về kỹ thuật phần mềm\n\n**Điểm mạnh của chương trình**\n\n* Kiến thức nền tảng vững chắc về kỹ thuật phần mềm\n* Kiến thức chuyên sâu về các lĩnh vực như phát triển ứng dụng, thiết kế giao diện, phân tích hệ thống, an toàn thông tin...\n* Cân bằng giữa lý thuyết và thực hành với nhiều tiết học thực 

## test retrievers

In [5]:
q = "em muốn biết thêm về ngành kỹ thuật phần mềm"
q = "hãy tư vấn ngành kỹ thuật phần mềm"
d = rag.bm25.get_relevant_documents(q)
d

[Document(page_content='Các học phần được thiết kế mang tính ứng dụng cao, theo hướng tăng dần từ cấp độ cơ bản đến cấp độ sinh viên có đủ kiến thức và kỹ năng để tự thực hiện được một dự án phần mềm, từ giai đoạn thu thập yêu cầu, phân tích, thiết kế, lập trình, kiểm thử đến giai đoạn vận hành và bảo trì phần mềm. Nhà trường còn đẩy mạnh chương trình kết nối mạng lưới doanh nghiệp thân hữu nhằm tạo điều kiện cho sinh viên có cơ hội tham quan và kiến tập từ năm học thứ 2. Bên cạnh đó, đối với một số môn học mang tính chuyên môn, giảng viên sẽ kết hợp với các chuyên gia dày dặn kinh nghiệm trực tiếp giảng dạy, góp phần bồi dưỡng thêm kiến thức và liiên tục cập nhật các tiêu chuẩn mới nhất về công nghệ để tăng khả năng cạnh tranh và hội nhập với nhu cầu ngày càng cao của xã hội.\n\nSinh viên ngành Kỹ thuật phần mềm sau khi tốt nghiệp sẽ được trang bị đầy đủ các kỹ năng như: kỹ năng phân tích, thiết kế, lập trình và kiểm thử các hệ thống phần mềm; kỹ năng triển khai và quản lý các dự án p

## test chain

In [8]:
q = "Trường có đạo tạo chuyên ngành về Việt Nam không ạ?"
q = "Cho em biết thông tin về ngành Dược ạ"
q = "em muốn biết thêm về ngành ỹ thuật phần mềm ạ"
a = rag.chain.invoke({"question": q})
aa

NameError: name 'aa' is not defined

In [10]:
a.answer

'None'

In [7]:
a.answer

'- Dược học là ngành học ứng dụng khoa học công nghệ để nghiên cứu về sự tương quan giữa thuốc với cơ thể người và cách sử dụng thuốc trong điều trị bệnh, đảm bảo chất lượng và phân phối thuốc cũng như cung cấp thông tin, hướng dẫn sử dụng thuốc cho cộng đồng.\n- Sinh viên sau khi tốt nghiệp được trang bị đầy đủ kiến thức và kỹ năng chuyên môn về quản lý, bào chế, sản xuất và lưu thông thuốc để có đủ tự tin hòa nhập vào đội ngũ y tế cộng đồng và tham gia vào tất cả các cơ sở hành nghề dược với các vị trí như: dược sĩ tư vấn sử dụng thuốc tại các nhà thuốc; dược sĩ nghiên cứu và phát triển thuốc của các công ty, xí nghiệp sản xuất thuốc tân dược, đông dược và sinh phẩm y tế, công ty xuất nhập khẩu và phân phối thuốc; chuyên viên kiểm nghiệm chất lượng và nguyên liệu làm thuốc tại các bệnh viện, trung tâm kiểm nghiệm của Trung ương, địa phương; dược sĩ cung ứng thuốc nội - ngoại trú bệnh viện; giảng viên giảng dạy chuyên ngành Dược tại các trường Đại học, …\n- Sinh viên theo học ngành Dư

In [12]:
@traceable(tags=['hybrid-rag', "debug-format"])
def run_rag(q):
    return rag.ask_rag(question=q)

q = "Cho em biết thông tin về ngành Dược ạ"
q = "Trường có đạo tạo chuyên ngành về Việt Nam không ạ?"

a = run_rag(q)
a

In [5]:
a.answer
a.document.source

'https://admission.tdtu.edu.vn/dai-hoc/nganh-hoc/viet-nam-hoc-chuyen-nganh-viet-ngu-hoc-va-van-hoa-xa-hoi-viet-nam'

In [7]:
a.document.content

'Việt Nam học - Chuyên ngành Việt ngữ học và văn hóa xã hội Việt Nam là một ngành khoa học nghiên cứu về đất nước, con người Việt Nam theo hướng tiếp cận liên ngành. Chương trình đào tạo ngành Việt Nam học, chuyên ngành Việt ngữ học và Văn hóa xã hội Việt Nam tại Trường Đại học Tôn Đức Thắng là chương trình dành cho đối tượng là người nước ngoài, nhằm đáp ứng nhu cầu về nguồn nhân lực là người nước ngoài sử dụng thông thạo tiếng Việt, am hiểu về đất nước và con người Việt Nam.\n\nSau khi tốt nghiệp ngành Việt Nam học, chuyên ngành Việt ngữ học và Văn hóa xã hội Việt Nam, học viên có thể tìm kiếm việc làm phù hợp ở các cơ quan ngoại giao, tổ chức quốc tế, công ty du lịch, văn phòng thương mại, tổ chức chính phủ và phi chính phủ có nhu cầu sử dụng nguồn nhân lực am hiểu về Việt Nam (ở Việt Nam và nước ngoài). Ngoài ra sinh viên cũng có thể làm công tác phiên dịch, biên dịch, nghiên cứu và giảng dạy tiếng Việt tại các trường đại học, cao đẳng hoặc các trung tâm ngôn ngữ trong và ngoài nướ

In [5]:
ans = rag.rag.invoke("Tư vấn về ngành CNTT trường Tôn Đức Thắng")
ans

'Tôi không tìm thấy thông tin về ngành CNTT trường Tôn Đức Thắng trong các văn bản được cung cấp.'

In [8]:
ans = rag.rag.invoke("Tư vấn về ngành CNTT")
ans

'- Khoa học máy tính là ngành ứng dụng kiến thức về thuật toán, phương pháp tính toán, ngôn ngữ lập trình, phương pháp phân tích và xây dựng hệ thống thông tin, … để giải quyết các vấn đề thuộc lĩnh vực Khoa học máy tính; phát triển các hệ thống phần mềm máy tính, mô hình tính toán thông minh, các ứng dụng trên các nền tảng hiện đại để phục vụ cho nhiệm vụ công nghiệp hoá, hiện đại hoá đất nước trong lĩnh vực chuyển đổi số.\n- Sau khi tốt nghiệp, cử nhân Khoa học máy tính có thể tự tin đảm nhận tốt các vị trí như chuyên viên thiết kế và xây dựng các phần mềm máy tính, ứng dụng cho thiết bị di động, ứng dụng thương mại điện tử trên nền Web, các trò chơi trên máy tính và di động; chuyên viên quản trị và xây dựng các giải pháp đảm bảo an toàn cho các hệ thống máy tính và hệ thống mạng máy tính; chuyên viên lập trình; chuyên viên phát triển hệ thống thông minh; chuyên viên tư vấn, thẩm định và phát triển các dự án, giải pháp công nghệ thông tin; cán bộ chuyên trách tại các cơ quan nhà nước

In [7]:
ans = rag.rag.invoke("Tư vấn về ngành Khoa học máy tính")
ans

'- Khoa học máy tính là ngành ứng dụng kiến thức về thuật toán, phương pháp tính toán, ngôn ngữ lập trình, phương pháp phân tích và xây dựng hệ thống thông tin, … để giải quyết các vấn đề thuộc lĩnh vực Khoa học máy tính; phát triển các hệ thống phần mềm máy tính, mô hình tính toán thông minh, các ứng dụng trên các nền tảng hiện đại để phục vụ cho nhiệm vụ công nghiệp hoá, hiện đại hoá đất nước trong lĩnh vực chuyển đổi số.\n- Sau khi tốt nghiệp, cử nhân Khoa học máy tính có thể tự tin đảm nhận tốt các vị trí như chuyên viên thiết kế và xây dựng các phần mềm máy tính, ứng dụng cho thiết bị di động, ứng dụng thương mại điện tử trên nền Web, các trò chơi trên máy tính và di động; chuyên viên quản trị và xây dựng các giải pháp đảm bảo an toàn cho các hệ thống máy tính và hệ thống mạng máy tính; chuyên viên lập trình; chuyên viên phát triển hệ thống thông minh; chuyên viên tư vấn, thẩm định và phát triển các dự án, giải pháp công nghệ thông tin; cán bộ chuyên trách tại các cơ quan nhà nước

In [4]:


docs = rag.ensemble_retriever.invoke("Tôi muốn học IT")

docs

[Document(page_content='Ngành Kỹ thuật Điện tử - Viễn thông là ngành học nghiên cứu từ những hệ thống hiện đại như các mạng thông tin di động thế hệ mới, công nghệ IoT, cho đến các thiết bị điện tử ứng dụng trong đời sống hàng ngày như mạch báo cháy, báo trộm, … nhằm phục vụ cho đời sống sinh hoạt và sự phát triển kinh tế của mỗi quốc gia.\nChương trình được thiết kế theo tiêu chuẩn kiểm định quốc tế, được đối sánh với các chương trình đại học uy tín trên thế giới và chuẩn đầu ra tin học đạt chứng chỉ tin học Microsoft Office Specialist tối thiểu 750/1000 điểm và kỹ năng ngoại ngữ đạt chứng chỉ tiếng Anh tối thiểu IELTS 5.0 hoặc chứng chỉ khác tương đương. Ngoài các kiến thức cơ bản về lý thuyết mạch điện - điện tử, kỹ thuật đo lường điện-điện tử, kỹ thuật mô phỏng bằng phần mềm, ngành học còn cung cấp cho sinh viên kiến thức về chuyên ngành như các phương pháp điều khiển kinh điển và hiện đại, các mạng truyền thông công nghiệp, công nghệ internet kết nối vạn vật, công nghệ robot, tự đ